# Random Forest Baseline

LL Train Test: 2.2538

LL Pred (On Kaggle): 

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

seed=16

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('../data/interim/training_meta_clean.csv')
df_pred = pd.read_csv('../data/interim/testing_meta_clean.csv')

In [3]:
df.describe()

,object_id,ra,decl,gal_l,gal_b,ddf,distmod,mwebv,target,hostgal_z
count,7.848000e+03,7848.000000,7848.000000,7848.000000,7848.000000,7848.000000,7848.000000,7848.000000,7848.000000,7848.000000
mean,4.810375e+07,169.719379,-27.254846,230.466784,-20.115526,0.269623,38.518322,0.081953,60.895387,0.254436
std,4.347364e+07,114.212761,20.248556,97.416392,40.390899,0.443792,4.638193,0.150598,28.767984,0.330053
min,6.150000e+02,0.175781,-64.760857,0.107681,-89.615573,0.000000,31.996100,0.003000,6.000000,0.000000
25%,3.180865e+05,62.402344,-44.597992,207.248509,-54.509752,0.000000,31.996100,0.018000,42.000000,0.000000
50%,4.131719e+07,150.996094,-27.279613,244.127187,-30.217582,0.000000,40.078500,0.032000,65.000000,0.182750
75%,8.665901e+07,282.875976,-8.838267,313.224277,16.913859,1.000000,41.877300,0.076000,90.000000,0.350125
max,1.307798e+08,359.824219,4.181528,359.943827,65.931317,1.000000,47.025600,2.747000,95.000000,3.445100


In [4]:
df_pred.describe()

,object_id,ra,decl,gal_l,gal_b,ddf,distmod,mwebv,hostgal_z
count,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06,3.492890e+06
mean,6.528181e+07,1.705897e+02,-2.624964e+01,2.178545e+02,-1.843280e+01,9.426578e-03,4.055387e+01,9.196644e-02,5.120204e-01
std,3.783131e+07,1.092068e+02,1.764381e+01,1.081992e+02,3.958926e+01,9.663188e-02,4.817608e+00,1.551435e-01,4.415488e-01
min,1.300000e+01,0.000000e+00,-6.476086e+01,1.036900e-02,-8.967441e+01,0.000000e+00,2.764620e+01,2.000000e-03,0.000000e+00
25%,3.252507e+07,7.277344e+01,-4.042390e+01,1.733004e+02,-5.280274e+01,0.000000e+00,4.067720e+01,2.300000e-02,2.664000e-01
50%,6.530507e+07,1.611914e+02,-2.511770e+01,2.459314e+02,-2.448392e+01,0.000000e+00,4.205530e+01,4.500000e-02,4.593000e-01
75%,9.805302e+07,2.758614e+02,-1.126298e+01,2.983127e+02,1.726843e+01,0.000000e+00,4.297340e+01,9.100000e-02,6.537000e-01
max,1.307881e+08,3.598242e+02,4.181528e+00,3.599956e+02,6.606870e+01,1.000000e+00,4.702610e+01,2.994000e+00,3.000000e+00


In [5]:
# Drop unneeded features
df = df.drop(['object_id', 'ddf'], axis=1)
test_object_id = df_pred['object_id']
df_pred = df_pred.drop(['object_id', 'ddf'], axis=1)

In [6]:
y = df['target']
X = df.drop(['target'], axis=1)

In [7]:
# Standardize
col_names = X.columns
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=col_names)
df_pred = pd.DataFrame(StandardScaler().fit_transform(df_pred), columns=col_names)

In [8]:
# Test, Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

In [9]:
rf = RandomForestClassifier(n_estimators=100, random_state=seed)
rf.fit(X_train, y_train)

test_scores = cross_val_score(rf, X_test, y_test, scoring='neg_log_loss', cv=5)
mean_test_cv_score = np.mean(test_scores)
std_dev_test_cv_score = np.std(test_scores)
        
train_scores = cross_val_score(rf, X_train, y_train, scoring='neg_log_loss', cv=5)
print("Mean Train CV Score: {0}".format(np.mean(train_scores)))
print("Std. Dev. Train CV Score: {0}".format(np.std(train_scores)))
print("Mean Test CV Score: {0}".format(mean_test_cv_score))
print("Std. Dev. Test CV Score: {0}".format(std_dev_test_cv_score)) 

Mean Train CV Score: -2.39209076553636
Std. Dev. Train CV Score: 0.10367608211330578
Mean Test CV Score: -2.253462611392343
Std. Dev. Test CV Score: 0.24240835090708737


In [10]:
for feature, importance in sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True):
    print(feature, importance)

hostgal_z 0.23623116804150382
distmod 0.1857385019142396
gal_b 0.12490409913891655
gal_l 0.11583665293963401
ra 0.11460195386331126
mwebv 0.11138291974479692
decl 0.11130470435759778


In [ ]:
preds = rf.predict_proba(df_pred)

In [ ]:
test_object_id = test_object_id.values.reshape((-1, 1))
class_99 = np.zeros((preds.shape[0], 1))
preds = np.hstack((test_object_id, preds, class_99))

output_cols = np.array(['object_id'] + ["class_" + str(x) for x in sorted(np.unique(y)) + [99]])

In [ ]:
preds = pd.DataFrame(preds, columns=output_cols)
preds['object_id'] = preds['object_id'].astype(int)

preds.to_csv('../data/submissions/rf_baseline.csv', index=False)